In [1]:
pip install -U vnstock

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=c51f1ebe8ad63d2a9b0fb4b20f5bcd71dbbb7be6878a39c674aa3e10d659dfa4
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install keras-self-attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=8074a9abccda0978e4b4c701000257fdddd14181ea29930f1b373f8c42e6b10f
  Stored in directory: /root/.cache/pip/wheels/46/f9/96/709295c836133071c12a300729fed4027757f889c01695feea
Successfully built keras-self-attention
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
from ta.momentum import RSIIndicator
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, RNN, concatenate
from tensorflow.keras.optimizers import RMSprop
from vnstock import *
from ta.volatility import BollingerBands
from tensorflow.keras.layers import Layer
import tensorflow as tf
from keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, Flatten, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
# from scikeras.wrappers import KerasClassifier

from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import CCIIndicator
from sklearn.preprocessing import StandardScaler
from ta.volume import ChaikinMoneyFlowIndicator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras import regularizers
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.saving import register_keras_serializable
import matplotlib.pyplot as plt
# from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from vnstock import *
import pandas as pd
import os
from collections import Counter
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier
from sklearn.metrics import accuracy_score # Import accuracy
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
import optuna
import joblib
from sklearn.metrics import (
    f1_score, accuracy_score, classification_report, roc_auc_score,
    roc_curve, auc
)
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import TimeSeriesSplit

2025-06-20 20:51:12.590039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750452672.785269      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750452672.840095      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [9]:
def dataframe_to_x_y(df, feature, target):
  X_train, y_train = [], []
  X_val, y_val = [], []
  n = len(df)
  split_index = int(n * 0.8)
  X_train = df[feature][:split_index]
  y_train = df[target][:split_index]
  X_val = df[feature][split_index:]
  y_val = df[target][split_index:]
  return X_train, y_train, X_val, y_val

In [10]:

sentiment_columns = [
        'Reputation', 'Company Communication', 'Appointment', 'Financial', 'Regulatory',
        'Sales', 'M&A', 'Legal', 'Dividend Policy', 'Risks', 'Rumors', 'Strategy',
        'Options', 'IPO', 'Signal', 'Coverage', 'Fundamentals', 'Insider Activity',
        'Price Action', 'Buyside', 'Technical Analysis', 'Trade', 'Central Banks',
        'Currency', 'Conditions', 'Market', 'Volatility', 'Investor Sentiment', 'Retail Investor Behavior',
        'Speculation', 'Domestic Institutional Behavior', 'Foreign Institutional Behavior', 'Black Swan Event',
    ]

sentiment_p1d_columns = [
        'reputation_p1d', 'company_communication_p1d', 'appointment_p1d', 'financial_p1d', 'regulatory_p1d',
        'sales_p1d', 'm&a_p1d', 'legal_p1d', 'dividend_policy_p1d', 'risks_p1d', 'rumors_p1d', 'strategy_p1d',
        'options_p1d', 'ipo_p1d', 'signal_p1d', 'coverage_p1d', 'fundamentals_p1d', 'insider_activity_p1d',
        'price_action_p1d', 'buyside_p1d', 'technical_analysis_p1d', 'trade_p1d', 'central_banks_p1d',
        'currency_p1d', 'conditions_p1d', 'market_p1d', 'volatility_p1d', 'investor_sentiment_p1d', 'retail_investor_behavior_p1d',
        'speculation_p1d', 'domestic_institutional_behavior_p1d', 'foreign_institutional_behavior_p1d', 'black_swan_event_p1d',
    ]
meta_cols = ['num_sa_news', 'mean_sentiment_norm', 'max_sentiment_norm', 'std_sentiment_score', 'sum_positive_sa', 'sum_negative_sa', 'time_distance_from_sa_to_close_minutes']
meta_p1d_cols = ['num_sa_news_p1d', 'mean_sentiment_norm_p1d', 'max_sentiment_norm_p1d', 'std_sentiment_score_p1d', 'sum_positive_sa_p1d', 'sum_negative_sa_p1d', 'time_distance_from_sa_to_close_minutes_p1d']

sentiment_feature_selected =  sentiment_columns + sentiment_p1d_columns + meta_cols + meta_p1d_cols
ta_features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','roc_1','roc_9','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','rsi_base_ma','rsi_rsi_base_ma_ratio','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','hl_ratio', 'co_ratio', 'price_range', 'sma_ratio_20_50', 'ema_ratio_12_26', 'bb_width', 'bb_position', 'rsi_overbought', 'rsi_oversold', 'rsi_neutral', 'macd_bullish', 'momentum_5', 'momentum_10','log_return','volatility_5d','volatility_10d','volatility_20d','volatility_30d','mean_log_return_5d','mean_log_return_10d','mean_log_return_20d','mean_log_return_30d','sharpe_like_5d','sharpe_like_10d','sharpe_like_20d','sharpe_like_30d','up_streak','pos_log_return_ratio_20d','z_score_5d','z_score_10d','z_score_20d','z_score_30d','annual_return','daily_return','sharpe_ratio',
               'rsi_vn30','rsi_base_ma_vn30','rsi_rsi_base_ma_ratio_vn30','volume_ma_vn30','volume_to_volume_ma_ratio_vn30','bb_bbm_vn30','bb_bbh_vn30','bb_bbl_vn30','bb_bbp_vn30','bb_bbh_bb_bbl_ratio_vn30','roc_1_vn30', 'roc_5_vn30', 'roc_9_vn30','%K_vn30','%R_vn30','cci_vn30','obv_vn30','ema_12_vn30','ema_26_vn30','sma_20_vn30','sma_50_vn30', 'hl_ratio_vn30', 'co_ratio_vn30', 'price_range_vn30', 'sma_ratio_20_50_vn30', 'ema_ratio_12_26_vn30', 'bb_width_vn30', 'bb_position_vn30', 'rsi_overbought_vn30', 'rsi_oversold_vn30', 'rsi_neutral_vn30', 'momentum_5_vn30', 'momentum_10_vn30', 'log_return_vn30','volatility_5d_vn30','volatility_10d_vn30','volatility_20d_vn30','volatility_30d_vn30','mean_log_return_5d_vn30','mean_log_return_10d_vn30','mean_log_return_20d_vn30','mean_log_return_30d_vn30','sharpe_like_5d_vn30','sharpe_like_10d_vn30','sharpe_like_20d_vn30','sharpe_like_30d_vn30','up_streak_vn30','pos_log_return_ratio_20d_vn30','z_score_5d_vn30','z_score_10d_vn30','z_score_20d_vn30','z_score_30d_vn30','annual_return_vn30','daily_return_vn30','sharpe_ratio_vn30',
               'rsi_vni','rsi_base_ma_vni','rsi_rsi_base_ma_ratio_vni','volume_ma_vni','volume_to_volume_ma_ratio_vni','bb_bbm_vni','bb_bbh_vni','bb_bbl_vni','bb_bbp_vni','bb_bbh_bb_bbl_ratio_vni','roc_1_vni', 'roc_5_vni', 'roc_9_vni','%K_vni','%R_vni','cci_vni','obv_vni','ema_12_vni','ema_26_vni','sma_20_vni','sma_50_vni', 'hl_ratio_vni', 'co_ratio_vni', 'price_range_vni', 'sma_ratio_20_50_vni', 'ema_ratio_12_26_vni', 'bb_width_vni', 'bb_position_vni', 'rsi_overbought_vni', 'rsi_oversold_vni', 'rsi_neutral_vni', 'momentum_5_vni', 'momentum_10_vni','log_return_vni','volatility_5d_vni','volatility_10d_vni','volatility_20d_vni','volatility_30d_vni','mean_log_return_5d_vni','mean_log_return_10d_vni','mean_log_return_20d_vni','mean_log_return_30d_vni','sharpe_like_5d_vni','sharpe_like_10d_vni','sharpe_like_20d_vni','sharpe_like_30d_vni','up_streak_vni','pos_log_return_ratio_20d_vni','z_score_5d_vni','z_score_10d_vni','z_score_20d_vni','z_score_30d_vni','annual_return_vni','daily_return_vni','sharpe_ratio_vni']
fa_features = ['p/b_previous_quarter', 'p/b_change_rate','p/b_change_rate_flag','p/e_previous_quarter','p/e_change_rate','p/e_change_rate_flag','p/s_previous_quarter','p/s_change_rate','p/s_change_rate_flag','p/cash_flow_previous_quarter','p/cash_flow_change_rate','p/cash_flow_change_rate_flag','eps_previous_quarter','eps_change_rate', 'eps_change_rate_flag','bvps_previous_quarter','bvps_change_rate', 'bvps_change_rate_flag','roe_previous_quarter','roe_change_rate','roe_change_rate_flag','roa_previous_quarter','roa_change_rate','roa_change_rate_flag','coefficient_p/b','coefficient_p/e','coefficient_p/s','coefficient_p/cash_flow','coefficient_eps','coefficient_bvps','coefficient_roe','coefficient_roa','distance_to_nearest_quarter']
ta_fa_feature_selected = ta_features + fa_features
features = ta_fa_feature_selected + sentiment_feature_selected
target = 'target'
scaler = StandardScaler()

In [11]:
def load_data_for_analysis(tickers, folder_path):

    df_all = []

    for ticker in tickers:
        file_path = os.path.join(folder_path, f"{ticker}.xlsx")
        if os.path.exists(file_path):
            df = pd.read_excel(file_path)
            df["ticker"] = ticker  # thêm cột ticker
            df_all.append(df)
        else:
            print(f"⚠️ Không tìm thấy: {file_path}")

    # Gộp lại thành 1 DataFrame
    df_global = pd.concat(df_all, ignore_index=True)
    print(f"✅ Đã load xong {len(df_global)} dòng dữ liệu từ {len(df_all)} cổ phiếu.")
    return df_global

In [12]:
tickers = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG',
                    'LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB',
                    'TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
folder_path_train = "/kaggle/input/vn30-dataset/v6 - Copy/out_performance_vn30/train_data" 
folder_path_test = "/kaggle/input/vn30-dataset/v6 - Copy/out_performance_vn30/test_data" 
df_global_train = load_data_for_analysis(tickers, folder_path_train)
df_global_test = load_data_for_analysis(tickers, folder_path_test)


✅ Đã load xong 35865 dòng dữ liệu từ 30 cổ phiếu.
✅ Đã load xong 7380 dòng dữ liệu từ 30 cổ phiếu.


In [13]:
# Danh sách các mô hình
models = {
    "Random Forest": RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced'),
    "XGBoost": XGBClassifier(n_jobs=-1, random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
    "LightGBM": LGBMClassifier(n_jobs=-1, random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
}

In [14]:
X_train_no_sa = df_global_train[ta_fa_feature_selected]
X_test_no_sa = df_global_test[ta_fa_feature_selected]
y_train = df_global_train[target]
y_test = df_global_test[target]
scaler = StandardScaler()
X_train_no_sa_scaled = scaler.fit_transform(X_train_no_sa)
X_test_no_sa_scaled = scaler.transform(X_test_no_sa) 

In [15]:
X_train_with_sa = df_global_train[features]  # features = ta + fa + sa
X_test_with_sa = df_global_test[features]
y_train = df_global_train[target]
y_test = df_global_test[target]

scaler = StandardScaler()
X_train_with_sa_scaled = scaler.fit_transform(X_train_with_sa)   # Fit scaler trên train, scale train luôn
X_test_with_sa_scaled = scaler.transform(X_test_with_sa)  

In [16]:
# ==============================================Tuning with Optuna + TimeSeriesSplit validate (AUC macro) ===============================================

Divide 5 fold to cross validation though tuning parameters

In [17]:
def get_grouped_time_series_folds(df, ticker_col='ticker', n_splits=5):
    tickers = df[ticker_col].unique()
    fold_indices = [[] for _ in range(n_splits)]  # List chứa n_splits fold tổng
    for ticker in tickers:
        idx = df[df[ticker_col] == ticker].index.values
        tscv = TimeSeriesSplit(n_splits=n_splits)
        for fold_num, (train_idx, val_idx) in enumerate(tscv.split(idx)):
            # Gộp index thực tế vào fold tổng tương ứng
            fold_indices[fold_num].append((idx[train_idx], idx[val_idx]))
    # Gộp index của từng fold tổng lại thành mảng train/val duy nhất
    grouped_folds = []
    for fold in fold_indices:
        train_idx = np.concatenate([item[0] for item in fold])
        val_idx = np.concatenate([item[1] for item in fold])
        grouped_folds.append((train_idx, val_idx))
    return grouped_folds

In [18]:
folds = get_grouped_time_series_folds(df_global_train, ticker_col='ticker', n_splits=5)

Tuning random forest for stock trend with ta + fa data

In [19]:
X_no_sa = X_train_no_sa_scaled  # hoặc X_train_with_sa_scaled nếu dùng cả SA
y = y_train

In [20]:
print("Tuning CatBoost with Optuna (AUC macro) ta + fa features only")

def objective_catboost(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 1200),
        "depth": trial.suggest_int("depth", 4, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "random_strength": trial.suggest_float("random_strength", 0, 2),
        "border_count": trial.suggest_int("border_count", 32, 255),
        # "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "auto_class_weights": "Balanced",  # giúp giảm mất cân bằng lớp
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "random_seed": 42,
        "task_type": "GPU",  # Nếu muốn dùng GPU trên Kaggle
        "devices": "0",      # Nếu chỉ dùng 1 GPU
        "verbose": 0,
    }

    aucs = []
    for train_idx, val_idx in folds:
        X_train, X_val = X_no_sa[train_idx], X_no_sa[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        model = CatBoostClassifier(**params)
        model.fit(X_train, y_train_fold)
        # One-hot encode y_val_fold
        y_val_bin = label_binarize(y_val_fold, classes=[0,1,2])
        y_pred_proba = model.predict_proba(X_val)
        # Nếu predict_proba trả về list (multioutput), cần stack lại
        if isinstance(y_pred_proba, list):
            y_pred_proba = np.stack([p[:,1] for p in y_pred_proba], axis=1)
        auc_macro = roc_auc_score(y_val_bin, y_pred_proba, average="macro", multi_class="ovr")
        aucs.append(auc_macro)
    mean_auc = np.mean(aucs)
    print(f"Trial {trial.number}: AUC macro={mean_auc:.5f}, params={trial.params}")
    # Optuna sẽ maximize AUC macro
    return -mean_auc

study = optuna.create_study(direction="minimize")
study.optimize(objective_catboost, n_trials=50)

print("Best AUC macro (CatBoost):", -study.best_value)
print("Best params (CatBoost):", study.best_trial.params)

[I 2025-06-20 20:54:22,318] A new study created in memory with name: no-name-5ea8c4b9-10d5-43c7-8ca0-8462db8ffc98


Tuning CatBoost with Optuna (AUC macro) ta + fa features only


[I 2025-06-20 20:56:11,140] Trial 0 finished with value: -0.5972710824656209 and parameters: {'iterations': 931, 'depth': 9, 'learning_rate': 0.043735995312967874, 'l2_leaf_reg': 5.3610954068933285, 'bagging_temperature': 0.2660573752345117, 'random_strength': 0.12120336749272753, 'border_count': 91}. Best is trial 0 with value: -0.5972710824656209.


Trial 0: AUC macro=0.59727, params={'iterations': 931, 'depth': 9, 'learning_rate': 0.043735995312967874, 'l2_leaf_reg': 5.3610954068933285, 'bagging_temperature': 0.2660573752345117, 'random_strength': 0.12120336749272753, 'border_count': 91}


[I 2025-06-20 20:59:56,766] Trial 1 finished with value: -0.6003092480111216 and parameters: {'iterations': 561, 'depth': 11, 'learning_rate': 0.0367258913867741, 'l2_leaf_reg': 4.2301199023016824, 'bagging_temperature': 0.33781285230266567, 'random_strength': 0.4021476573579079, 'border_count': 230}. Best is trial 1 with value: -0.6003092480111216.


...

Trial 47: AUC macro=0.60912, params={'iterations': 803, 'depth': 8, 'learning_rate': 0.010922731596068064, 'l2_leaf_reg': 6.92955930009963, 'bagging_temperature': 0.8461521446198097, 'random_strength': 1.8478642578777436, 'border_count': 204}


[I 2025-06-20 22:00:39,155] Trial 48 finished with value: -0.6008701239426827 and parameters: {'iterations': 735, 'depth': 6, 'learning_rate': 0.04516490751777353, 'l2_leaf_reg': 6.958836093773484, 'bagging_temperature': 0.9045538521787722, 'random_strength': 1.5092378266774267, 'border_count': 237}. Best is trial 42 with value: -0.6102446609504362.


Trial 48: AUC macro=0.60087, params={'iterations': 735, 'depth': 6, 'learning_rate': 0.04516490751777353, 'l2_leaf_reg': 6.958836093773484, 'bagging_temperature': 0.9045538521787722, 'random_strength': 1.5092378266774267, 'border_count': 237}


[I 2025-06-20 22:01:45,733] Trial 49 finished with value: -0.6079651823695136 and parameters: {'iterations': 691, 'depth': 8, 'learning_rate': 0.02342050155436142, 'l2_leaf_reg': 8.872035597555186, 'bagging_temperature': 0.8284561906331612, 'random_strength': 1.307503064638906, 'border_count': 251}. Best is trial 42 with value: -0.6102446609504362.


Trial 49: AUC macro=0.60797, params={'iterations': 691, 'depth': 8, 'learning_rate': 0.02342050155436142, 'l2_leaf_reg': 8.872035597555186, 'bagging_temperature': 0.8284561906331612, 'random_strength': 1.307503064638906, 'border_count': 251}
Best AUC macro (CatBoost): 0.6102446609504362
Best params (CatBoost): {'iterations': 611, 'depth': 7, 'learning_rate': 0.017772855089424312, 'l2_leaf_reg': 5.417401265075917, 'bagging_temperature': 0.9927068493506941, 'random_strength': 1.5271877954308244, 'border_count': 234}


Tuning random forest for stock trend with ta + fa + sa data

In [21]:
X_with_sa = X_train_with_sa_scaled  
y = y_train

In [22]:
print("Tuning CatBoost with Optuna (AUC macro) ta + fa + sa features")

def objective_catboost(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 500, 1500),
        "depth": trial.suggest_int("depth", 5, 14),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 12),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "random_strength": trial.suggest_float("random_strength", 0, 2),
        "border_count": trial.suggest_int("border_count", 32, 255),
        # "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "auto_class_weights": "Balanced",  # giúp giảm mất cân bằng lớp
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "random_seed": 42,
        "task_type": "GPU",   # Sử dụng GPU trên Kaggle
        "devices": "0",       # Chọn GPU đầu tiên
        "verbose": 0,
    }

    aucs = []
    for train_idx, val_idx in folds:
        X_train, X_val = X_with_sa[train_idx], X_with_sa[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        model = CatBoostClassifier(**params)
        model.fit(X_train, y_train_fold)
        # One-hot encode y_val_fold
        y_val_bin = label_binarize(y_val_fold, classes=[0,1,2])
        y_pred_proba = model.predict_proba(X_val)
        # Nếu predict_proba trả về list (multioutput), cần stack lại
        if isinstance(y_pred_proba, list):
            y_pred_proba = np.stack([p[:,1] for p in y_pred_proba], axis=1)
        auc_macro = roc_auc_score(y_val_bin, y_pred_proba, average="macro", multi_class="ovr")
        aucs.append(auc_macro)
    mean_auc = np.mean(aucs)
    print(f"Trial {trial.number}: AUC macro={mean_auc:.5f}, params={trial.params}")
    # Optuna sẽ maximize AUC macro
    return -mean_auc

study = optuna.create_study(direction="minimize")
study.optimize(objective_catboost, n_trials=50)

print("Best AUC macro (CatBoost):", -study.best_value)
print("Best params (CatBoost):", study.best_trial.params)

[I 2025-06-20 22:01:45,759] A new study created in memory with name: no-name-8c2837a5-5fa1-4651-b937-c363f31a9d26


Tuning CatBoost with Optuna (AUC macro) ta + fa + sa features


[I 2025-06-20 22:03:08,921] Trial 0 finished with value: -0.5912936334037041 and parameters: {'iterations': 1383, 'depth': 7, 'learning_rate': 0.11958069076617905, 'l2_leaf_reg': 7.748927618002314, 'bagging_temperature': 0.613124511201712, 'random_strength': 0.1154573071219942, 'border_count': 155}. Best is trial 0 with value: -0.5912936334037041.


Trial 0: AUC macro=0.59129, params={'iterations': 1383, 'depth': 7, 'learning_rate': 0.11958069076617905, 'l2_leaf_reg': 7.748927618002314, 'bagging_temperature': 0.613124511201712, 'random_strength': 0.1154573071219942, 'border_count': 155}


[I 2025-06-20 22:07:48,767] Trial 1 finished with value: -0.601486813566123 and parameters: {'iterations': 1144, 'depth': 10, 'learning_rate': 0.0702795138432888, 'l2_leaf_reg': 1.8037877189554807, 'bagging_temperature': 0.0811641870308869, 'random_strength': 1.2569783348209598, 'border_count': 254}. Best is trial 1 with value: -0.601486813566123.


...

Trial 47: AUC macro=0.61341, params={'iterations': 946, 'depth': 8, 'learning_rate': 0.024021250845348563, 'l2_leaf_reg': 5.271056385748106, 'bagging_temperature': 0.9448362605467473, 'random_strength': 0.14149266347800005, 'border_count': 138}


[I 2025-06-21 01:32:39,812] Trial 48 finished with value: -0.6156895847376987 and parameters: {'iterations': 873, 'depth': 6, 'learning_rate': 0.010385197327040435, 'l2_leaf_reg': 4.512739746314608, 'bagging_temperature': 0.830881574308519, 'random_strength': 0.2757644318983876, 'border_count': 111}. Best is trial 42 with value: -0.6164444978363374.


Trial 48: AUC macro=0.61569, params={'iterations': 873, 'depth': 6, 'learning_rate': 0.010385197327040435, 'l2_leaf_reg': 4.512739746314608, 'bagging_temperature': 0.830881574308519, 'random_strength': 0.2757644318983876, 'border_count': 111}


[I 2025-06-21 01:33:20,424] Trial 49 finished with value: -0.615898413888278 and parameters: {'iterations': 867, 'depth': 6, 'learning_rate': 0.010769079193189392, 'l2_leaf_reg': 3.9283506473928242, 'bagging_temperature': 0.8287216177065587, 'random_strength': 0.3238605723003105, 'border_count': 112}. Best is trial 42 with value: -0.6164444978363374.


Trial 49: AUC macro=0.61590, params={'iterations': 867, 'depth': 6, 'learning_rate': 0.010769079193189392, 'l2_leaf_reg': 3.9283506473928242, 'bagging_temperature': 0.8287216177065587, 'random_strength': 0.3238605723003105, 'border_count': 112}
Best AUC macro (CatBoost): 0.6164444978363374
Best params (CatBoost): {'iterations': 995, 'depth': 8, 'learning_rate': 0.014141428456986765, 'l2_leaf_reg': 5.540109008031062, 'bagging_temperature': 0.8775942516652349, 'random_strength': 0.009807881231376103, 'border_count': 91}
